## Hands-on session 1.3 - Fourier heat law

This notebook aims to familiarise you with several important features used often in constructing ReMKiT1D simulations. 

A number of cells are left blank or unfinished as an exercise.

Demonstrated concepts:

- Stationary variables
- Setting Dirichlet boundary conditions by excluding cells from operators
- Using the diffusion stencil for variables on the regular grid
- Calculation tree derivations
- Dashboard function for comparing multiple fluid variables

In [1]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Basic setup

In [2]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_1_3/"
rk.setHDF5Path(hdf5Filepath) 

#MPI setup
rk.setMPIData(numProcsX=4)

#Grid initialization
xGridWidths = 0.025*np.ones(512)
gridObj = Grid(xGridWidths, interpretXGridAsWidths=True)
rk.grid = gridObj

### The equations

We wish to solve the following well known equation

$$\frac{\partial T}{\partial t} = - \frac{\partial q}{\partial x}$$

$$q = -\kappa \frac{\partial T}{\partial x}$$

With some boundary conditions $T(x=x_0)=T_1$ and $T(x=x_N)=T_2$

Here we ignore any normalization issues and simply work in arbitrary units.

### Variables

We seem to have 2 equations, but only one time derivative. We could substitute $q$ into the equation for $T$ and get a second order equation, or we could have two first order equations. 

Add the variables (and their dual variables) for two evolved temperatures "T_diff" and "T_q", to be used for the two approaches mentioned above. 

The initial value of the temperature is supplied below

In [3]:
# Initial values, as well as the two boundary values for temperature
T = np.ones(512)
T[0] = 10 
T[-1] = 5

# This adds both the var and its dual automatically along with the initial values.
# Rule of thumb is to set isCommunicated=True esp. if variable is under differential operators
rk.addVarAndDual("T_q", T, isCommunicated=True)
rk.addVarAndDual("T_diff", T, isCommunicated=True)

To add a variable that does not have a time derivative in its equation, we simply mark it as `stationary` by setting the `isStationary` argument to `addVar` or `addVarAndDual`.

Add a stationary variable for the heat flux 'q'. Since we are still working on a staggered grid, and 'q' represents a flux, we should make sure that `primaryOnDualGrid` is set to `True`:

In [4]:
# If you don't pass an initial condition it defaults to 0.
rk.addVarAndDual("q", np.zeros(512), primaryOnDualGrid=True, isCommunicated=True, isStationary=True)

And finally the `time` variable

In [5]:
rk.addVar("time",isScalar=True,isDerived=True)

### Calculation tree derivations

A particularly convenient way of producing derivation rules for ReMKiT1D, which is applicable in many situations, is to translate Python-like expressions into a tree represenation. 

In order to generate one such object, we first need to declare any variables we wish to use as Nodes. At the beginning of this notebook `Node` has been imported directly from ReMKiT1D. 

Let's suppose we have the following familiar expression for the conductivity $\kappa= \kappa_0 T^{3/2}$. Which grid do you think this should be evaluated on?

Add the appropriate variable name ('T_diff' or 'T_diff_dual') to the Node constructor below

In [6]:
# This T is solved as part of the q equation which is solving for q on the edges (dual  grid).
nodeT = Node("T_diff_dual") # <- YOUR CODE HERE

kappa0 = 0.01
nodeKappa = kappa0 * nodeT**(3/2)

We can then manually add this derivation to the wrapper (we will need it for the diffusion stencil!)

In [7]:
rk.addCustomDerivation("kappa_diff",treeDerivation(nodeKappa)) 

Let's also add a variable for $\kappa$, but using the other temperature 'T_q'. We can do it in an elegant 'one-liner', defining the corresponding derivation rule as the variable is added

In [8]:
# Again, we are solving for q which is on the dual grid, so must use T on the dual grid.
# Variables need communication unless they exist in one core and don't need to be anywhere else,
# or if they are implicit variables only and live in the matrix.

# Uncomment the one you think is correct!

kappaOnDualGrid = True
# kappaOnDualGrid = False

rk.addVar("kappa_q",
          isDerived=True,
          derivOptions=treeDerivation(kappa0*Node("T_q_dual")**(3/2)), # <- YOUR CODE HERE (HINT: Which variable do we need to calculate kappa)
          derivationRule=sc.derivationRule("kappa_q",requiredVars=["T_q_dual"]), # <- YOUR CODE HERE (HINT: Which variable do we need to calculate kappa)
          isCommunicated=True,
          isOnDualGrid=kappaOnDualGrid)

The above will add the variable 'kappa_q', together with a derivation named "kappa_q" based on the `derivOptions` argument. 

The `requiredVars` argument to `derivationRule` is there to make sure that the needed temperature is calculated before 'kappa_q'.

### On Dirichlet boundary conditions

ReMKiT1D by default does not use ghost cells to set Dirichlet boundary conditions. Instead, as the reader might have surmised, we have taken the first and the last cell in the domain and set them to some values. But how do we avoid evolving those cells?

The answer is in the `spatialProfile` argument to the `GeneralMatrixTerm` construct:

In [9]:
help(sc.GeneralMatrixTerm)

Help on class GeneralMatrixTerm in module RMK_support.simple_containers:

class GeneralMatrixTerm(builtins.object)
 |  GeneralMatrixTerm(evolvedVar: str, implicitVar='', spatialProfile: List[float] = [], harmonicProfile: List[float] = [], velocityProfile: List[float] = [], evaluatedTermGroup=0, implicitGroups=[1], generalGroups=[1], customNormConst: Union[RMK_support.simple_containers.CustomNormConst, float, int] = <RMK_support.simple_containers.CustomNormConst object at 0x7f72045529a0>, timeSignalData=<RMK_support.simple_containers.TimeSignalData object at 0x7f7204552a60>, varData=<RMK_support.simple_containers.VarData object at 0x7f7204552ac0>, stencilData={}, skipPattern=False, fixedMatrix=False, copyTermName: Union[str, NoneType] = None) -> None
 |  
 |  General (custom) matrix term options used to construct custom models
 |  
 |  Methods defined here:
 |  
 |  __init__(self, evolvedVar: str, implicitVar='', spatialProfile: List[float] = [], harmonicProfile: List[float] = [], veloc

The default profile, if the user does not supply one, is simply all ones. We will however, replace it by the following:

In [10]:
# This is a straight up multiplier on a variable and will be used to set the diffusion coefficient
# to zero in the final cell. The domain terminates on the final cell centre (?)

spatialProfile = np.ones(512)
spatialProfile[0] = spatialProfile[-1] = 0
spatialProfile = spatialProfile.tolist() # Since the constructor requires a list! 

We will use this profile below for both models we're adding.

### The diffusion stencil approach

A prebuilt diffusion stencil is available using the following function

In [11]:
sc.diffusionStencil?

Signature:
sc.diffusionStencil(
    ruleName: str,
    reqVarNames: List[str],
    doNotInterpolate=False,
) -> dict
Docstring:
Return diffusion stencil in x which assumes that both the implicit and evolved variables live on the regular grid.

Args:
    ruleName (str): Name of derivation used to calculate the diffusion coefficent
    reqVarNames (List[str]): Variable names required for the derivation
    doNotInterpolate (bool, optional): If true will assume that the rule already calculates the diffusion coefficient on cell boundaries. Defaults to False.
Returns:
    dict: Stencil property dictionary
File:      /usr/local/lib/python3.8/dist-packages/RMK_support/simple_containers.py
Type:      function

As it can be seen, we need a defined derivation rule in order to use it. Fortunately, we have already added the custom derivation 'kappa_diff'.

The diffusion stencil corresponds to the operator $\nabla \kappa \nabla$, with the $\kappa$ associated with the derivation rule passed to the stencil.

Note that the above documentation requires both the evolved and implicit variable to live on the regular grid for this stencil to be correct (This will be caught by the Fortran code).

In [12]:
rk.varList()

['T_q', 'T_q_dual', 'T_diff', 'T_diff_dual', 'q_dual', 'q', 'time', 'kappa_q']

In [13]:
newModel = sc.CustomModel("fourierDiff")

# My kappa lives on the cell edges already, which means if I provide a stencil based on it I have to tell it to not interpolate onto edges.
# Remember that the diffusion stencil assumes everything is on the cell centres by default!

# Here we are directly evolving T by using the diffusion equation, so we are using evolvedVar = T_diff. 
# The implicit variable is also T_diff.

diffusionTerm = sc.GeneralMatrixTerm(evolvedVar="T_diff", # <- YOUR CODE HERE (Which variable are we evolving?)
                                     implicitVar="T_diff", # <- YOUR CODE HERE (Which variable is the diffusion operator acting on?)
                                     spatialProfile=spatialProfile,
                                     stencilData=sc.diffusionStencil("kappa_diff", ["T_diff_dual"], 
                                                                     doNotInterpolate = True)) # YOUR CODE HERE (Complete the diffusion stencil call based on the documentation - HINT: Which variable(s) is/are required and where do they live?) 

newModel.addTerm("diffTerm",diffusionTerm)

rk.addModel(newModel)

Checking terms in model fourierDiff:
   Checking term diffTerm


### Stationary $q$ approach

Often the second order operator is not directly available, unlike with our diffusion example. In those situations we can either build a custom stencil, or resort to multiple first order equations. 

We begin by rewriting the $q$ equation so that all terms are on the RHS (further emphasizing the lack of a time derivative)

$$0 = -q -\kappa \nabla T$$

We see that we will need to add two terms for the q equation, including an identity term for $q$:

In [14]:
newModel = sc.CustomModel("fourierQ")

identityTerm = sc.GeneralMatrixTerm(evolvedVar="q_dual", # When no implicit variable is supplied it defaults to the evolved variable
                                    customNormConst=-1,
                                    stencilData=sc.diagonalStencil()) # A diagonal stencil, effectively the Kronecker symbol

newModel.addTerm("q_identity",identityTerm)

Let's add the temperature gradient term. We're evolving 'q_dual', which lives on the staggered grid. We want to take the gradient of a temperature variable. 

In [15]:
# Need to use the staggeredGradStencil cause we're using a staggered grid.
# You can also use central differencing but only if both variables are implicit..

gradTerm = sc.GeneralMatrixTerm(evolvedVar="q_dual",
                                implicitVar="T_q", # <- YOUR CODE HERE (Which variable are we applying the gradient to?)
                                customNormConst=-1,
                                varData=sc.VarData(reqRowVars=["kappa_q"]),
                                stencilData=sc.staggeredGradStencil()) # <- YOUR CODE HERE (The stencilData argument is missing, which one should it be?)

newModel.addTerm("gradT",gradTerm)

Finally, we need the divergence of the heat flux in the temperature equation (remember, we have equations for both the temperature and the heat flux!)

In [16]:
# We are evolving temperature in the first equation of the split method, i.e. T_q.
# The implicit variable used to calculate is q which lives on the dual grid.
# Remember that implicit T is on centres but implicit q is on the edges. Their 
# edge/centre equivalents are derived and so are not implicit!

divQ = sc.GeneralMatrixTerm(evolvedVar="T_q", # <- YOUR CODE HERE (Which variable is this term evolving?)
                            implicitVar="q_dual", # <- YOUR CODE HERE (Which variable are we applying the divergence to? Which grid does it live on?)
                            customNormConst=-1,
                            spatialProfile=spatialProfile,# Applying the same spatial profile as in the diffusion stencil approach
                            stencilData=sc.staggeredDivStencil()) # <- YOUR CODE HERE (The stencilData argument is missing, which one should it be?) 

newModel.addTerm("divQ",divQ)

rk.addModel(newModel)

Checking terms in model fourierQ:
   Checking term q_identity
   Checking term gradT
   Checking term divQ


### Time integration options

Following the approach from the previous sessions, we set the following:

In [17]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=["T_q", "T_diff"])) # <- YOUR CODE HERE (We want all evolved variables here, which ones are they?)

rk.setIntegratorGlobalData(initialTimestep=0.1) 

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Write config file

In [18]:
rk.writeConfigFile()

### Set global plotting options

In [19]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

### Load data from ReMKiT1D output files

In [20]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

<xarray.Dataset>
Dimensions:      (x: 512, h: 1, v: 1, time: 51)
Coordinates:
  * x            (x) float64 0.0125 0.0375 0.0625 0.0875 ... 12.74 12.76 12.79
  * h            (h) int64 0
  * v            (v) float64 1.0
  * time         (time) float64 0.0 20.0 40.0 60.0 ... 940.0 960.0 980.0 1e+03
Data variables:
    T_q          (time, x) float64 10.0 1.0 1.0 1.0 ... 5.054 5.036 5.018 5.0
    T_q_dual     (time, x) float64 5.5 1.0 1.0 1.0 ... 5.045 5.027 5.009 4.991
    T_diff       (time, x) float64 10.0 1.0 1.0 1.0 ... 5.054 5.036 5.018 5.0
    T_diff_dual  (time, x) float64 5.5 1.0 1.0 1.0 ... 5.045 5.027 5.009 4.991
    q_dual       (time, x) float64 0.0 0.0 0.0 0.0 ... 0.08151 0.08151 0.0
    q            (time, x) float64 0.0 0.0 0.0 0.0 ... 0.08151 0.08151 0.08151
    kappa_q      (time, x) float64 0.129 0.01 0.01 0.01 ... 0.1127 0.1121 0.1115

### Explore data with built-in dashboard

In [21]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


Launching server at http://localhost:38001


/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7f721915c3a0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x7f721915c3a0> will be resolved to its underlying value :Curve   [time]   (T_q). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(callable=callable,
/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7f72190b31f0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <functi

We can also explore multiple fluid variables on a single graph using another dashboard function

In [ ]:
dashboard.fluidMultiComparison(["T_diff","T_q"]) #Note: this is a HoloMap object so doesn't have the same show() method as the dashboard above